# 🧹 Churn Prediction – Data Preprocessing Notebook
이 노트북은 `shop.db`에서 데이터를 불러와 고객 이탈 예측용 Feature Dataset을 생성합니다.


In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
import os
print('Imports 완료')

## 1. DB 로드

In [ ]:
conn = sqlite3.connect('shop.db')
cursor = conn.cursor()
print('DB 연결 완료')

## 2. 고객별 주문 집계

In [ ]:
query = '''
SELECT
    o.customer_id,
    COUNT(o.order_id) AS total_orders,
    SUM(oi.quantity * oi.unit_price) AS total_amount,
    AVG(oi.quantity * oi.unit_price) AS avg_amount,
    MIN(o.order_date) AS first_order,
    MAX(o.order_date) AS last_order
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_status = 'paid'
GROUP BY o.customer_id;
'''
df = pd.read_sql_query(query, conn)
df

## 3. Feature Engineering

In [ ]:
df['first_order'] = pd.to_datetime(df['first_order'])
df['last_order'] = pd.to_datetime(df['last_order'])
df['days_between'] = (df['last_order'] - df['first_order']).dt.days
df['avg_days_per_order'] = df['days_between'] / df['total_orders'].replace(0,1)
df

## 4. Churn Label 생성 (1회 구매 고객 = 1)

In [ ]:
df['churn_label'] = df['total_orders'].apply(lambda x: 1 if x==1 else 0)
df[['customer_id','total_orders','churn_label']].head()

## 5. CSV 저장

In [ ]:
os.makedirs('data', exist_ok=True)
df.to_csv('data/churn_dataset.csv', index=False)
print('data/churn_dataset.csv 저장 완료')